In [144]:
'''
    Code in this Python Jupyter Notebook has been obtained from:
    http://api.mongodb.com/python/current/tutorial.html
'''
import pymongo
from pymongo import MongoClient

In [145]:
# Create a MongoClient to interact with MongoDB Service
client = MongoClient()

In [146]:
# List all database names in the connected server
client.list_database_names()

['admin', 'config', 'local', 'profile_database', 'test_database']

In [147]:
# Choose a database
db = client['test_database']

In [148]:
# List all collections in a database
db.list_collection_names()

['posts', 'test_collection']

In [149]:
# Choose a collection
collection = db['test_collection']

In [150]:
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test_database'), 'test_collection')

# Documents

Data in MongoDB is represented (and stored) using JSON-style documents. In PyMongo we use dictionaries to represent documents. As an example, the following dictionary might be used to represent a blog post:

In [151]:
import datetime
post = {"author": "Mike",      
            "text": "My first blog post!",
            "tags": ["mongodb", "python", "pymongo"],
            "date": datetime.datetime.utcnow()}

## Inserting a Document
To insert a document into a collection we can use the insert_one() method:

In [152]:
posts = db.posts # Accessing a different collection named 'posts'
posts.delete_many({}) # Clears the entire table 'posts'

post_id = posts.insert_one(post).inserted_id
post_id

ObjectId('5d2b8b59a92f7b7479bcb38e')

When a document is inserted a special key, "_id", is automatically added if the document doesn’t already contain an "_id" key. The value of "_id" must be unique across the collection. insert_one() returns an instance of InsertOneResult. For more information on "_id", see the documentation on _id.

In [153]:
db.list_collection_names()

['posts', 'test_collection']

## Getting a Single Document With find_one()
The most basic type of query that can be performed in MongoDB is find_one(). This method returns a single document matching a query (or None if there are no matches). It is useful when you know there is only one matching document, or are only interested in the first match. Here we use find_one() to get the first document from the posts collection:

In [154]:
import pprint
pprint.pprint(posts.find_one())

{'_id': ObjectId('5d2b8b59a92f7b7479bcb38e'),
 'author': 'Mike',
 'date': datetime.datetime(2019, 7, 14, 20, 6, 49, 205000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


find_one() also supports querying on specific elements that the resulting document must match. To limit our results to a document with author “Mike” we do:

In [155]:
pprint.pprint(posts.find_one({"author": "Mike"}))

{'_id': ObjectId('5d2b8b59a92f7b7479bcb38e'),
 'author': 'Mike',
 'date': datetime.datetime(2019, 7, 14, 20, 6, 49, 205000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


## Querying By ObjectId
We can also find a post by its _id, which in our example is an ObjectId:

In [156]:
pprint.pprint(posts.find_one({"_id": post_id}))

{'_id': ObjectId('5d2b8b59a92f7b7479bcb38e'),
 'author': 'Mike',
 'date': datetime.datetime(2019, 7, 14, 20, 6, 49, 205000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


## Bulk Inserts
In order to make querying a little more interesting, let’s insert a few more documents. In addition to inserting a single document, we can also perform bulk insert operations, by passing a list as the first argument to insert_many(). This will insert each document in the list, sending only a single command to the server:

In [157]:
new_posts = [{"author": "Mike",
               "text": "Another post!",
               "tags": ["bulk", "insert"],
               "date": datetime.datetime(2009, 11, 12, 11, 14)},
              {"author": "Eliot",
               "title": "MongoDB is fun",
               "text": "and pretty easy too!",
               "date": datetime.datetime(2009, 11, 10, 10, 45)}]
result = posts.insert_many(new_posts)

In [158]:
result.inserted_ids

[ObjectId('5d2b8b59a92f7b7479bcb38f'), ObjectId('5d2b8b59a92f7b7479bcb390')]

## Querying for More Than One Document
To get more than a single document as the result of a query we use the find() method. find() returns a Cursor instance, which allows us to iterate over all matching documents. For example, we can iterate over every document in the posts collection:

In [159]:
for post in posts.find():
    pprint.pprint(post)

{'_id': ObjectId('5d2b8b59a92f7b7479bcb38e'),
 'author': 'Mike',
 'date': datetime.datetime(2019, 7, 14, 20, 6, 49, 205000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}
{'_id': ObjectId('5d2b8b59a92f7b7479bcb38f'),
 'author': 'Mike',
 'date': datetime.datetime(2009, 11, 12, 11, 14),
 'tags': ['bulk', 'insert'],
 'text': 'Another post!'}
{'_id': ObjectId('5d2b8b59a92f7b7479bcb390'),
 'author': 'Eliot',
 'date': datetime.datetime(2009, 11, 10, 10, 45),
 'text': 'and pretty easy too!',
 'title': 'MongoDB is fun'}


We can filter as we did earlier:

In [160]:
for post in posts.find({"author": "Mike"}):
    pprint.pprint(post)

{'_id': ObjectId('5d2b8b59a92f7b7479bcb38e'),
 'author': 'Mike',
 'date': datetime.datetime(2019, 7, 14, 20, 6, 49, 205000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}
{'_id': ObjectId('5d2b8b59a92f7b7479bcb38f'),
 'author': 'Mike',
 'date': datetime.datetime(2009, 11, 12, 11, 14),
 'tags': ['bulk', 'insert'],
 'text': 'Another post!'}


## Counting
If we just want to know how many documents match a query we can perform a count_documents() operation instead of a full query. We can get a count of all of the documents in a collection:

In [161]:
posts.count_documents({})

3

or just of those documents that match a specific query:

In [162]:
posts.count_documents({"author": "Mike"})

2

## Range Queries
MongoDB supports many different types of advanced queries. As an example, lets perform a query where we limit results to posts older than a certain date, but also sort the results by author:

In [163]:
d = datetime.datetime(2009, 11, 12, 12)
for post in posts.find({"date": {"$lt": d}}).sort("author"):
    pprint.pprint(post)

{'_id': ObjectId('5d2b8b59a92f7b7479bcb390'),
 'author': 'Eliot',
 'date': datetime.datetime(2009, 11, 10, 10, 45),
 'text': 'and pretty easy too!',
 'title': 'MongoDB is fun'}
{'_id': ObjectId('5d2b8b59a92f7b7479bcb38f'),
 'author': 'Mike',
 'date': datetime.datetime(2009, 11, 12, 11, 14),
 'tags': ['bulk', 'insert'],
 'text': 'Another post!'}


Here we use the special "$lt" operator to do a range query, and also call sort() to sort the results by author.